# Notes on Conversion of pwmig to parallel_pwmig

March 12, 2021

This notebook is my working notes describing how I converted the original pwmig package to the revised version
I'm (tentatively) calling parallel_pwmig.  This is an extensive conversion because the revision will use the 
MsPASS framework.  There is a related directory in docs/design that contains preliminary thoughts on how this 
conversion be done. 

## Initial repository setup
The first thing I needed to do was assemble components from other directories.  Because pwmig originally had a dependency on antelope that involved moving some code from antelope contrib.   I also copied files from the original pwmig repository.  The files are a complete reorganization in directories from the parent code but there were no name changes.  Some files were not copied that were already in mspass or I knew were unnecessary.
I checked these into the repository master branch to preserve the entire history chain of the code.  

I then used the mspass cmake configuration files in include and source directories to set up the 
cmake system.  To get started I just built the root and linking Cmake file that define how cmake 
works through the directory chain.  The reason is that right now I'm no positive what will be converted
to python and what will be left as C++ main programs.   Also the correct library structure isn't clear 
either.  Hence, I'll kick that can down the road.  I'll concentrate instead on some of the initial code hacking.
Not sure how much of that I will actually record here.   It is the usual think of symbol changes, changes 
to include filel names, and configuration errors.   Warning is what is recorded below will likely be 
incomplete, but will try to record high points.  I'll make notes on each source directory.   

**Hypocenter**.   Quickly discovered I missed copying coords and stock libraries I had adapted for pwmig from 
the public domain version of antelope (Datascope).   Copied that to utility areas.  Removed the elog.h and 
elog.c file that we don't want her.  elog calls are going to be generic problem I will need to fix.  The seispp version of hypocenter had travel time methods embedded in the class.  That was a dumb design 
that I removed in the conversion.  I did, however, retain convenience methods for computing distance, 
seaz, and esaz.   NOTE:  this class might end up moving into MsPASS.

A good thing came from this in converting this function.  I used the keywords.h file to remove the literal 
key names used previously to fetch lat,lon, depth, and origin time in Metadata driven constructors.

**EventCatalog**.   The main reason I needed to convert Hypocenter was to use this C++ class that 
was used in telecluster to subset events into radial grid segments.   The implementation used does 
not translate easily at all to python and would be compute intensive if done with a dumber algorithm.
The original EventCatalog used Antelopes database to read event data.  It had two constructors 
using an antelope database handle.   For this application using pymongo I am removing those constructors
and depending on the use of append methods.  This will drastically slow construction as it will require 
a python load but this should not be an issue for telecluster as the number of teleseisms of use for 
this kind of processing is currently not that enormous relative to current computer memory sizes.  Certainly not something to worry about until it proves to be a problem. 

Note did a lot of interface changes to make proper use of const. 

**gclgrid**  This is a much bigger job due to the shear size of this library.   I debated a big reorganization and/or cleaning up problems with the design (this was my first major c++ library) but decided that was counterproductiv.  

For the rest of this I will write down the key stages in the modifications.

1.  The include changes needed in all of these files.
2.  Tedious job of using const appropriately in all the classes.  At the same time I cleared all the sections 
    within the ifndefs for use with antelope.  This created long list of compilation issues dealing with 
    const mismatches.   The dark side of fixing that kind of thing.
3.  Working through this made me realize an obvious gap of a program that needed conversion:  makegclgrid. 
    Added a new design page for that program.
4.  Another thing I cleared was GCLgrid_error was depricated for GCLgridError that was set up as a child of 
    std::exception
5.  Added a constructor for all classes with one arg; his generic signature:  GCLgridtype(const Metadata&)
    All depend on having valid dir, dfile, and foff entries in the metadata container.
6.  Decided to use a namespace pwmig::gclgrid.   That created another lengthy set of compilation errors 
    that had to be handled.  Note this and the const changes were both not essential for a conversion to 
    mspass but something I elected to do since I was going through the whole code base anyway.
7.  Forgot about a name change.  Seispp had a PfStyleMetadata that in mspass became AntelopePf with a fairly 
    significant change in the api.  
8.  extract_component functions now throw a GCLgridError instead of using an elog and throw int that I thought 
    had been previously depricated.  I think these functions are actually not used anywhere in pwmig.
    
Overall there were a lot of changes to even get this library to compiles.

**EventCatalog**  Just one file, but there were many issues.

1.  First had to add the line to add ${MSPASSHOME}/include to CMakeList.txt file as with gclgrid.
2.  A bunch of relic SEISPP stuff that had to be cleared. 
3.  A bunch more const related errors I created by putting that in the conversion.

**Hypocenter**  This one was pretty easy.   Some of the same issues as other library stuff.  One thing of note about this file is that I fundamental changed the API for the Hypocenter object.  It previously contained a lot of travel time related methods.  I realized that was an unnecessary complication for a lower level class.  I considered, in fact, a redesign with an abstract base and and extended Hypocenter with travel times but judged it unnecessary at this time.  

March 17, 2021
At this stage most of the library stuff has been converted to the point of at least compiling.  Pushed this version to github before moving to the next step.

Next issue is pybind11 bindings for library functions.   I don't want to just convert everything but will add pybind11 hooks as I need them to support what I need to move into python.  I'll document that in this file in the box below with the title "Initial Python Bindings".

## Build System

March 15-16, 2021

After initial changes to pwmig for the mspass framework I realized I needed a mechanism to more effectivly use the build system with cmake.  mspass installs compiled shared libraries and installs them and include files in a area defined by MSPASSHOME.  Seeking help from Ian on how to do his more automatically.   The fallback is to just require the MSPASSHOME environment variable.   Will update this whe I get an answer.

There may be a way to automatically do a cmake package someday but for now am going the environmeent variable route.  To do that I did the following:
1.  Had to build the full C libraries in mspass with the :code:`make_install` method.  That required setting an 
    environment variable MSPASS_HOME to a writeable directory.  
2.  In the top level CMakeList.txt file I added a line to extract MSPASS_HOME from the environment and set 
    a slight variant as the cmake variable MSPASSHOME.   All directories that need mspass include files need
    a ${MSPASSHOME} inserted appropriately (gclgrid was the first example I found that worked.)   

## Reorganization

*May 19, 2021*

In hacking to get the cmake/make system to build this package I ended up doing a major reorganization of the source code directories.   I had initially tried to structure things similar to the old pwmig repository.  I realized, however, that the structure of those files was weird because of I was trying to maintain a common code base for the antelope contrib source code and pwmig.   With mspass that is not longer relevant as the mspass code base has evolved far from the original antelope code base.  Hence, I build a new structure that is somewhat but not identically parallel to mspass.  Basically I created four lib libraries in four directories:
1. **dsap** contains all old datascope source code transferred from the public domain version of antelope that was called Datascope (dsap).
2.  **gclgrid** the old gclgrid directory with byte swap routines added.
3.  **pwmigcore** pwmig specific library functions.   Initially small but I expect to push some other code there eventually. 
4.  **seispp** conversion of Antelope seispp functions for use with mspass not in mspass.  Initially that is two classes:  Hypocenter and EventCatalog.  It also includes the son of Morozov's C interpolation routines that were previously in a pwmig only.   Putting these in seispp is a bit inconsistent wrt to origin but the functionality is more appropriate there are they are utility functions.

That reduced a lot of the mess I had trying to build this package, but am still left with a weird thing I cannot figure out.  The pybind11 section is failing to link propertly because something in the came cmake configuration is inserting "-lutility".   That creates a not found, but figuring out what"utility" that means is very ambiguoius.  Punting this to Ian for help.


# Initial Python Bindings

## telecluster

*May 18, 2021*

The first program I need to implement into a python framework is telecluster.  I need that because it has to interact with mongodb to get data from a source collection.  Perusing the C++ code I realize these are needed:

1.  The key algorithm in that program used a template function of EventCatalog where the template uses a 
    C++ class defined in telecluster called SectorTest.  The rest of the code is little more than a wrapper 
    around that class to (a) load the EventCatalog with data and (b) loop through a radial grid finding 
    events inside each radial sector and saving them. 
2.  I think this can be made a command line tool that acts exactly like telecluster.  It can probably even 
    match the arg signature but will use the simpler tools to crack the command line in python.  I'll 
    borrow from my recent experience with dbverify in mspass.

So initial python bindings are needed for:
1. Hypocenter objects
2. EventCatalog objects
3. Need to move the RadialGrid and SectorTest code from telecluster.cc to lib and build required bindings. 

Will do that in that order.  Hypocenter is a good starting point as the bindings are pretty simple. 

*May 23, 2021*
Had to learn a lot more about cmake to get all this working, but finally sorted out a library problem I was having to get all the pieces built and generate a valid so file for python with pybind11 for the seispp module.   

To support telecluster I added (a modified version) of code for two special classes that were previously part of telecluster:  (1) RadialGrid (what it sound like) and (2) SectorTest - a weird predicate class used for the subset operation of EventCatalog.  

Building a new telecluster function defined within a file called telecluster.py that should have pretty much the same functionality as the old telecluster c++ program.  I decided to test it on the usarraytest database as that had an appropriately large collection of earthquake sources (6000+).  I quickly discovered, however, that that database was old and had some naming problems.  I decided this was a good way to test the new clean functions Weiming had written for mspass.  The next blocks shows how I did that.  First verify the size and create the database handles.

In [1]:
from mspasspy.db.database import Database
from mspasspy.db.client import Client
dbclient=Client()
db=Database(dbclient,'usarraytest')
source_size=db.source.count_documents({})
print('source collection size of usarraytest database = ',source_size)

source collection size of usarraytest database =  9756


So is is more like 10000 events.  This next block shows what the issue is with the data there:

In [2]:
doc=db.source.find_one()
print(doc.keys())

dict_keys(['_id', 'source_lat', 'source_lon', 'source_depth', 'source_time', 'magnitude', 'magnitude_type', 'serialized_event', 'source_id'])


This db was generated a while back when the Database schema stuff was incomplete and the source_ stuff needs to be stripped.  The new clean method is supposed to do that.   Here is the run to do that.

In [5]:
# this is how clean defines a rename operation
rnmap={ 'source_lat' : 'lat', 'source_lon' : 'lon', 'source_depth' : 'depth', 'source_time' : 'time'}
db.clean_collection('source',rename_undefined=rnmap)

{}

Let's check that it did what it should have done:

In [6]:
doc=db.source.find_one()
print(doc.keys())

dict_keys(['_id', 'lat', 'lon', 'depth', 'time', 'magnitude', 'magnitude_type', 'serialized_event', 'source_id'])


Discovered that this database had another wart.  Some of the source documents had 'latitude' and 'longitude' as keys.  Easily repaired the same way:

In [7]:
rnmap={'latitude' : 'lat', 'longitude' : 'lon'}
db.clean_collection('source',rename_undefined=rnmap)

{}

With this I got my proto script mostly working.  I had to make a fix that revealed something important for mspass though.  I had seen before in pybind11's documentation that shared_ptr wrapped objects needed some care.  Indeed the subset method I moved from telecluster seg faulted until I changed the api to return a copy instead of the EventCatalog it was filling instead of a shared_ptr to an EventCatalog object.  I think this is deep in the ugly realm of memory management mismatch between C and python.

With that fix I was able to get a working new python version of telecluster.   It produces a new "telecluster" collection.  Each document in the *cluster* collection has these attributes:

1.  *hypocentroid* key links to a dict that creates a subdocument in mongodb. The attributes in the subdocument are the same as a source document record for the coordinates;  lat, lon, depth, and time.  The time is largely meaningless but may be useful to avoid the made up lrigin times I used in the previous version of pwmig. 
2.  *events* keys a list of source_id's converted to strings that are the events in source located inside a particular radial grid cell.
3.  *cell* keys a dict with attributes defining the corners of the box that define this grid cell:  deltamin, deltamax, azmin, azmax.  It also contains two integers keyed with *azindex* and *deltaindex* that are the original RadialGrid cell indices.   